In [1]:
!pip install -r https://raw.githubusercontent.com/google-ai-edge/ai-edge-torch/main/requirements.txt
!pip install ai-edge-torch-nightly

  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached torch-2.6.0-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached torch_xla2-0.0.1.dev202412041639-py3-none-any.whl.metadata (2.5 kB)
  Using cached scipy-1.15.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached jax-0.6.0-py3-none-any.whl.metadata (22 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.meta

In [5]:
!git clone https://github.com/mit-han-lab/proxylessnas.git

Cloning into 'proxylessnas'...


/usr/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


remote: Enumerating objects: 1815, done.
remote: Counting objects: 100% (182/182), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 1815 (delta 111), reused 74 (delta 58), pack-reused 1633 (from 1)
Receiving objects: 100% (1815/1815), 266.46 MiB | 7.30 MiB/s, done.
Resolving deltas: 100% (543/543), done.
Updating files: 100% (1360/1360), done.


In [1]:
from proxyless_nas.model_zoo import proxyless_mobile
net = proxyless_mobile()  # returns a ready-to-train nn.Module

Downloading: "https://raw.githubusercontent.com/han-cai/files/master/proxylessnas/proxyless_mobile.config" to /home/zclawrence/.torch/proxyless_nas/proxyless_mobile.config
Downloading: "https://raw.githubusercontent.com/han-cai/files/master/proxylessnas/proxyless_mobile.pth" to /home/zclawrence/.torch/proxyless_nas/proxyless_mobile.pth


In [2]:
import torch
from torch import nn

class ProxylessAudioClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.model = proxyless_mobile()

        # Modify input layer for 1-channel input (Mel spectrogram)
        self.model.first_conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )

        # Modify output classifier
        self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [11]:
import ai_edge_torch
import torch
import tensorflow as tf

NUM_CLASSES = 6

model = ProxylessAudioClassifier(NUM_CLASSES)
model.load_state_dict(torch.load("./buow_proxylessnas.pth", weights_only=True))
sample_inputs = (torch.randn(1, 1, 64, 259),)

#TFLite Quantization: https://www.tensorflow.org/lite/performance/model_optimization
tfl_converter_flags = {'optimizations': [tf.lite.Optimize.DEFAULT]}

# Convert and serialize PyTorch model to a tflite flatbuffer. Note that we
# are setting the model to evaluation mode prior to conversion.
edge_model = ai_edge_torch.convert(model.eval(), sample_inputs, _ai_edge_converter_flags=tfl_converter_flags)
edge_model.export("./buow_proxylessnas_quantized.tflite")

INFO:tensorflow:Assets written to: /tmp/tmphrmtqsgp/assets


INFO:tensorflow:Assets written to: /tmp/tmphrmtqsgp/assets
W0000 00:00:1747037760.635912   12921 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1747037760.635981   12921 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-05-12 01:16:00.636173: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmphrmtqsgp
2025-05-12 01:16:00.640368: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-05-12 01:16:00.640381: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmphrmtqsgp
2025-05-12 01:16:00.687551: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-05-12 01:16:01.034356: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmphrmtqsgp
2025-05-12 01:16:01.116023: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 479853

In [ ]:
#Verifying that inference results are within tolerable range between the original torch model and the converted, quantized tflite model

import numpy as np

torch_output = model(*sample_inputs)
edge_output = edge_model(*sample_inputs)
print(torch_output.detach().numpy())
print(edge_output)
print(np.allclose(torch_output.detach().numpy(), edge_output, atol=1e-1))

[[-1.2254138  -6.0098476  -0.15234268 -2.717904   -2.495277    0.5122019 ]]
[[-1.2193803  -6.042346   -0.16378047 -2.718736   -2.5040097   0.5123157 ]]
True


In [26]:
!pip install ai-edge-model-explorer

/usr/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 MB 30.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [ai-edge-model-explorer]ge-model-explorer]


In [ ]:
#The script below is for visualizing the model graph, not too sure how it works, but the API mentioned it so I thought I would include it here too

In [27]:
!model-explorer './buow_proxylessnas_quantized.tflite'

/usr/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Loading extensions...
Loaded 8 extensions:
 - TFLite adapter (Flatbuffer)
 - TFLite adapter (MLIR)
 - TF adapter (MLIR)
 - TF adapter (direct)
 - GraphDef adapter
 - Pytorch adapter (exported program)
 - MLIR adapter
 - JSON adapter

Starting Model Explorer server at:
http://localhost:8080/?data=%7B%22models%22%3A%20%5B%7B%22url%22%3A%20%22/mnt/c/Users/zachl/OneDrive/Desktop/CSE%20145_2/proxylessnas/buow_proxylessnas_quantized.tflite%22%7D%5D%7D

Press Ctrl+C to stop.
gio: http://localhost:8080/?data=%7B%22models%22%3A%20%5B%7B%22url%22%3A%20%22/mnt/c/Users/zachl/OneDrive/Desktop/CSE%20145_2/proxylessnas/buow_proxylessnas_quantized.tflite%22%7D%5D%7D: Operation not supported
Stopping server...
^C
